<a href="https://colab.research.google.com/github/Loop3D/2022_iamg_workshop/blob/main/map2loop/Utility_1_Config_file_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# hjson Config file generator


# Don't use this notebook!!! Go to https://github.com/Loop3D/qgis-loopplugin


> Indented block


This simple notebook provides a tool for generating map2loop config hjson files and a minimum python script, one of the more annoying parts of the process.
   
1. It first gets the project name and destination projection system (as EPSG number)   
   
2. It then asks for the paths to the different shape files (for WFS you will have to edit this manually afterwards)   
   
3. Next for each input file, you have to define which fields to use, and which keywords to look for in some of these fields 

4. Finally you can alter the user definable parameters
     
5. This generates a minimum python script for your data with all the paths and parameters hopefully correctly configured.This can be run as is or pasted into a new notebook. 


In [ ]:
!git clone https://github.com/Loop3D/map2loop2-notebooks
!pip install ipyfilechooser
!pip install git+https://github.com/geopandas/geopandas.git@v0.10.2

## Select project name & coordinate reference system

In [ ]:
%cd map2loop2-notebooks
import utils_m2l as um

c_l,epsg= um.init_codes()
test_data_name_choice,crs_choice=um.display_general(epsg)


## Select Bounding Box

In [ ]:
test_data_name,dst_crs=um.get_general(test_data_name_choice,crs_choice)
minx_choice,miny_choice,maxx_choice,maxy_choice,minz_choice,maxz_choice=um.display_3D_bbox()

## Select Input File paths

In [ ]:
bbox_3D=um.get_bbox_3D(minx_choice,miny_choice,maxx_choice,maxy_choice,minz_choice,maxz_choice)
structure_file_m,fault_file_m,fold_file_m,geology_file_m,mindep_file_m,dtm_file_m=um.display_filepaths()

## Connect  Structral parameter names

In [ ]:
structure_file,fault_file,fold_file,geology_file,mindep_file,dtm_file=um.get_filepaths(structure_file_m,fault_file_m,fold_file_m,geology_file_m,mindep_file_m,dtm_file_m)
gi_choice,sf_choice,bedding_choice,d_choice,dd_choice,otype_choice,bo_choice,btype_choice=um.display_structure_fields(structure_file)

## Connect Fault parameter names

In [ ]:
c_l=um.get_structure_fields(c_l,gi_choice,sf_choice,bedding_choice,d_choice,dd_choice,otype_choice,bo_choice,btype_choice)
f_choice,fault_choice,fdip_choice,fdipnull_choice,fdipdir_choice,fdipdir_flag_choice,fdipest_choice=um.display_fault_fields(fault_file)

## Connect  Geology parameter names

In [ ]:
c_l=um.get_fault_fields(c_l,f_choice,fault_choice,fdip_choice,fdipnull_choice,fdipdir_choice,fdipdir_flag_choice,fdipest_choice)
o_choice,c_choice,g_choice,g2_choice,u_choice,ds_choice,r1_choice,r2_choice,sill_choice,intrusive_choice,volcanic_choice,min_choice,max_choice=um.display_geology_fields(geology_file)


## Define map2loop runtime control parameter names

In [ ]:
c_l=um.get_geology_fields(c_l,o_choice,c_choice,g_choice,g2_choice,u_choice,ds_choice,r1_choice,r2_choice,sill_choice,intrusive_choice,volcanic_choice,min_choice,max_choice)
aus_choice,deposits_choice,intrusion_mode_choice,dtb_choice,orientation_decimate_choice,contact_decimate_choice,\
            misorientation_choice,interpolation_scheme_choice,\
            fault_decimate_choice,min_fault_length_choice,\
            fault_dip_choice,pluton_dip_choice,pluton_form_choice,dist_buffer_choice,\
            contact_dip_choice,close_dip_choice,use_interpolations_choice,\
            use_fat_choice,clut_path_choice,fault_orientation_clusters_choice,fault_length_clusters_choice,use_roi_clip_choice,roi_clip_path_choice,\
            contact_orientation_decimate_choice,fat_step_choice,fold_decimate_choice,interpolation_spacing_choice,\
            max_thickness_allowed_choice,thickness_buffer_choice=um.display_m2l_config()

## Save out config file and example python script file

In [ ]:
run_flags,clut_path=um.get_m2l_config(aus_choice,deposits_choice,intrusion_mode_choice,dtb_choice,orientation_decimate_choice,contact_decimate_choice,
            misorientation_choice,interpolation_scheme_choice,
            fault_decimate_choice,min_fault_length_choice,
            fault_dip_choice,pluton_dip_choice,pluton_form_choice,dist_buffer_choice,
            contact_dip_choice,close_dip_choice,use_interpolations_choice,
            use_fat_choice,clut_path_choice,fault_orientation_clusters_choice,fault_length_clusters_choice,use_roi_clip_choice,roi_clip_path_choice,
            contact_orientation_decimate_choice,fat_step_choice,fold_decimate_choice,interpolation_spacing_choice,
            max_thickness_allowed_choice,thickness_buffer_choice)

um.save_hjson(c_l,test_data_name)
um.save_m2l_run_script(run_flags,bbox_3D,clut_path,test_data_name,dst_crs,geology_file,fault_file,fold_file,structure_file,mindep_file,dtm_file)

## You can paste the python script ***_run_m2l.py into the next cell to try it out   
# Need Roy to add back shapefiles, and find out how to disable mindeps and folds

In [ ]:
from map2loop.project import Project
from map2loop.m2l_enums import VerboseLevel

proj = Project(
                geology_filename='/home/mark/map2loop2-notebooks/source_data/geol_clip.shp',
                fault_filename='/home/mark/map2loop2-notebooks/source_data/faults_clip.shp',
                fold_filename='/home/mark/map2loop2-notebooks/source_data/faults_clip.shp',
                structure_filename='/home/mark/map2loop2-notebooks/source_data/structure_clip.shp',
                mindep_filename='./source_data/mindeps_clip.shp',
                dtm_filename='http://services.ga.gov.au/gis/services/DEM_SRTM_1Second_over_Bathymetry_Topography/MapServer/WCSServer?',
                metadata_filename='./Test/Test_config.hjson',
                overwrite='true',
                verbose_level=VerboseLevel.NONE,
                project_path='./Test_results',
                working_projection='epsg:28350',
                )

proj.update_config(
                    out_dir='./Test_results',
                    bbox_3d={'minx': 520000.0, 'miny': 7490000.0, 'maxx': 550000.0, 'maxy': 7510000.0, 'base': -3200.0, 'top': 1200.0},
                    run_flags={'aus': True, 'close_dip': -999.0, 'contact_decimate': 5, 'contact_dip': -999.0, 'contact_orientation_decimate': 5, 'deposits': 'Fe,Cu,Au,NONE', 'dist_buffer': 10.0, 'dtb': '', 'fat_step': 750.0, 'fault_decimate': 5, 'fault_dip': 90.0, 'fold_decimate': 5, 'interpolation_scheme': 'scipy_rbf', 'interpolation_spacing': 500.0, 'intrusion_mode': 0, 'max_thickness_allowed': 10000.0, 'min_fault_length': 5000.0, 'misorientation': 30.0, 'null_scheme': 'null_scheme', 'orientation_decimate': 0, 'pluton_dip': 45.0, 'pluton_form': 'domes', 'thickness_buffer': 5000.0, 'use_fat': False, 'use_interpolations': False, 'fault_orientation_clusters': 2, 'fault_length_clusters': 2, 'use_roi_clip': False, 'roi_clip_path': ''},
                    proj_crs='epsg:28350',
                    clut_path= '',
                )

proj.run()
